# Query MLFlow to find the latest production model

This is a simple demo as presented in https://databricks.com/session_na20/accelerating-mlflow-hyper-parameter-optimization-pipelines-with-rapids.
It simply pulls out the latest "production" model from the MLFlow model registry and uses it directly within Python. See also the mlflow_project dierctory for more detailed MLFlow production examples.

In [2]:
import mlflow
from pprint import pprint

model_name = "MLFlow_Airline_RAPIDS"
model_id = None
model_uri = None

client = mlflow.tracking.MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    if (mv.current_stage == "Production"):
        run_data = client.get_run(mv.run_id)
        model_uri = mv.source
        model_id = mv.run_id
        break

print(f"Model: {model_uri}")

Model: dbfs:/databricks/mlflow/4049571819107025/caa94ab8152141908e4869ee33ce8574/artifacts/RF_model_GPU

## Load the model back to memory

In [4]:
import mlflow.sklearn
model = mlflow.sklearn.load_model(model_uri)
print(model.__class__)

<class 'cuml.ensemble.randomforestclassifier.RandomForestClassifier'>

# Finally load data and predict

In [6]:
import cudf

df = cudf.read_parquet("/dbfs/data/airlines/airline_20000000.parquet")
X = df[df.columns.difference(["ArrDelayBinary"])]
# X = df.drop("ArrDelayBinary", axis=1)
model.predict(X)

Out[30]: 0 0.0
1 0.0
2 1.0
3 0.0
4 0.0
 ... 
19999995 0.0
19999996 0.0
19999997 0.0
19999998 0.0
19999999 0.0
Length: 20000000, dtype: float32

In [7]:
X.columns

Out[25]: Index(['ActualElapsedTime', 'CRSArrTime', 'CRSDepTime', 'DayofMonth',
 'DayofWeek', 'Dest', 'Distance', 'Diverted', 'FlightNum', 'Month',
 'Origin', 'UniqueCarrier', 'Year'],
 dtype='object')

In [8]:
df.drop("ArrDelayBinary", axis=1).columns

Out[26]: Index(['Year', 'Month', 'DayofMonth', 'DayofWeek', 'CRSDepTime', 'CRSArrTime',
 'UniqueCarrier', 'FlightNum', 'ActualElapsedTime', 'Origin', 'Dest',
 'Distance', 'Diverted'],
 dtype='object')

In [9]:
df.columns

Out[27]: Index(['ArrDelayBinary', 'Year', 'Month', 'DayofMonth', 'DayofWeek',
 'CRSDepTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum',
 'ActualElapsedTime', 'Origin', 'Dest', 'Distance', 'Diverted'],
 dtype='object')